<img src="../imgs/Adevinta-ULPGC-logo.jpg" width="530px" align="right">

# **Aprendizaje por refuerzo 3**

Todo lo que hemos visto en los apuntes anteriores corresponden a soluciones basadas en [programación dinámica]( https://es.wikipedia.org/wiki/Programación_dinámica). Su principal característica es que son solamente aplicables cuando se conoce *a priori* todo el entorno en el que el agente se va a desenvolver. Es decir, conocemos cuáles son las recompensas en cada estado y las probabilidades de transición entre estados. Como imaginarás, esto no es siempre posible en situaciones reales, por tanto, necesitamos desarrollar métodos que busquen políticas óptimas mediante la **exploración** y la **exploración**. Veremos los métodos basados en estrategias de Monte Carlo y basadas en diferencias temporales.

## **Monte Carlo**

En la **Programación Dinámica** es necesario conocer de antemano el modelo del entorno, es decir, las probabilidades de transición $P_{s,s'}^a$ y las recompensas $R_{s,s'}^a$, para poder evaluar y mejorar las políticas. Sin embargo, en situaciones donde el modelo del entorno no es conocido, podemos utilizar los métodos de **Monte Carlo**. Estos métodos evalúan una política observando las recompensas que el agente recibe al interactuar con el entorno. Esto requiere realizar múltiples **episodios** desde diferentes **estados iniciales** para obtener una **estimación del retorno esperado** desde los distintos estados.

En cada iteración del algoritmo Monte Carlo se ejecuta un episodio completo de acuerdo a la política actual y, posteriormente, se actualiza la estimación del retorno basándose en los resultados obtenidos. El objetivo es aprender la política óptima, lo cual puede lograrse a través de la estimación de la **función de valor** $V(s)$ o de la **función acción-valor** $Q(s,a)$.

#### **Actualización de Políticas**

La actualización de la política utilizando la función de valor se expresa como sigue:

$$\pi_{k+1}(s) = \arg \max_a \left( \sum_{s'} P_{s,s'}^a \left[ R_{s,s'}^a + \gamma \cdot V^\pi_k(s') \right] \right)$$

Mientras que la actualización utilizando la función acción-valor se define de la siguiente manera:

$$\pi_{k+1}(s) = \arg \max_a Q_k(s,a)$$

En los métodos de Monte Carlo, se prefiere usar la función acción-valor $Q(s,a)$, ya que no requiere almacenar el modelo de comportamiento del entorno ($P_{s,s'}^a$ y $R_{s,s'}^a$).


#### **Algoritmo Monte Carlo para estimar $Q(s,a)$**

1. **Generación de episodios**: Simular un episodio siguiendo la política $\pi$, registrando estados, acciones, recompensas y retornos.
2. **Actualización de $Q(s,a)$**:
   - Para cada estado $s$ visitado en el episodio:
     - $R$: Retorno obtenido desde el estado $s$.
     - $a$: Acción tomada en el estado $s$.
     - Agregar $R$ a la lista de retornos para el par estado-acción $(s, a)$.
     - $Q(s,a) = \text{Promedio de retornos para } (s,a)$.
3. **Repetición**: Repetir los pasos 1 y 2.

Dado que se puede pasar múltiples veces por el mismo estado en un episodio, existen dos formas de estimar la función acción-valor:
- Utilizar tan sólo la primera ocurrencia de cada estado (*first-visit*)
- Utilizar todas las ocurrencias del estado (*every-visit*).

Se puede demostrar que la primera versión converge a la estimación correcta de la función acción-valor. La segunda opción aprovecha más información de cada episodio, pero su convergencia es más lenta.

## **Diferencias Temporales**

En los métodos de **Monte Carlo**, aprender la política óptima no requiere conocer el comportamiento del entorno pero sí la realización de episodios completos para actualizar la estimación de $V(s)$ o $Q(s,a)$. En cambio, la **programación dinámica** actualiza estos valores basándose en los estados vecinos sin necesidad de episodios completos, aunque depende de conocer el modelo del entorno. El aprendizaje por **diferencias temporales** (TD) combina elementos de ambos métodos: permite la actualización de la función de valor $V(s)$ de los estados visitados basándose tanto en las recompensas obtenidas como en los valores de los estados vecinos, similar a la programación dinámica, mientras que los episodios se desarrollan siguiendo alguna política $\pi$, similar al método Monte Carlo.

El algoritmo de diferencias temporales más básico, conocido como **TD(0)**, actualiza los valores de estado mediante la siguiente fórmula:

$$
V(s) = V(s) + \alpha [r + \gamma V(s') - V(s)]
$$

Esta ecuación tiene un significado profundo: $V(s) - \gamma V(s')$ representa la diferencia esperada de valor entre el estado actual $s$ y el siguiente estado $s'$, mientras que $r - (V(s) - \gamma V(s'))$ es la diferencia entre la recompensa observada y la recompensa esperada en la transición de $s$ a $s'$. Por tanto, la actualización de $V(s)$ se basa en estas diferencias, ajustando la estimación del valor del estado en función de la experiencia directa y la expectativa previa.

Para asegurar la convergencia del algoritmo, es crítico incluir un factor de aprendizaje $\alpha$, que debe ser menor que 1. Este factor no solo ayuda a la convergencia sino que también permite al algoritmo adaptarse a cambios en el entorno, haciendo posible que el agente maneje entornos no estacionarios, donde las probabilidades de transición y las recompensas pueden variar con el tiempo.

#### **Implementación del Algoritmo TD(0)**

El algoritmo se puede describir en pasos concretos dentro de un bucle que continúa hasta que se alcanza la convergencia:

```python
while not convergencia:
    inicializar_episodio()
    while episodio_en_curso:
        accion = politica(s)  # Acción elegida según la política en el estado s
        recompensa, siguiente_estado = ejecutar_accion(accion)  # Recompensa y nuevo estado tras la acción
        V[s] = V[s] + alpha * (recompensa + gamma * V[siguiente_estado] - V[s])
        s = siguiente_estado
```

Este enfoque iterativo no solo refina continuamente la estimación del valor de cada estado visitado durante el episodio, sino que también ajusta dinámicamente las expectativas del agente a medida que adquiere más experiencia del entorno.

## **SARSA (State–Action–Reward–State–Action)**

Este algoritmo es similar al anterior, pero haciendo uso de la **función acción-valor** $Q(s,a)$ en lugar de la **función valor** $V(s)$. En SARSA, la actualización de los valores Q depende tanto del estado y acción actual, como del siguiente estado y la acción que se tomará a continuación, lo que le da un carácter "on-policy", es decir, la política que se sigue es la misma que se mejora.

La fórmula de actualización en SARSA es:

$$
Q(s,a) = Q(s,a) + \alpha [r + \gamma Q(s',a') - Q(s,a)]
$$

Donde:
- $ \alpha $ es el factor de aprendizaje.
- $ r $ es la recompensa recibida al transicionar del estado $ s $ al estado $ s' $ mediante la acción $ a $.
- $ \gamma $ es el factor de descuento, que pondera la importancia de las recompensas futuras.
- $ Q(s',a') $ es el valor estimado para la siguiente acción $ a' $ en el siguiente estado $ s' $, según la política actual.

#### **Implementación del algoritmo SARSA**

El algoritmo SARSA sigue un enfoque iterativo, como se describe a continuación:

```python
while not convergencia:
    inicializar_episodio()
    accion = politica(estado_actual)  # Selección de la acción según la política
    while episodio_en_curso:
        recompensa, siguiente_estado = ejecutar_accion(accion)  # Ejecución de la acción
        siguiente_accion = politica(siguiente_estado)  # Determinación de la siguiente acción
        Q[estado_actual, accion] = Q[estado_actual, accion] + alpha * (recompensa + gamma * Q[siguiente_estado, siguiente_accion] - Q[estado_actual, accion])
        estado_actual = siguiente_estado
        accion = siguiente_accion
```


## **Q-Learning**

A diferencia de SARSA, Q-learning adopta un enfoque "off-policy", lo que significa que no está atado a la política actual para sus actualizaciones. En Q-learning, se busca maximizar las recompensas futuras de manera más agresiva al escoger siempre el máximo valor de $Q$ en el siguiente estado, sin importar la política actual.

La fórmula de actualización en Q-learning es:

$$
Q(s,a) = Q(s,a) + \alpha [r + \gamma \max_{a'}(Q(s',a')) - Q(s,a)]
$$

### **Implementación del algoritmo Q-Learning**

El proceso iterativo para Q-learning se puede describir de la siguiente manera:

```python
while not convergencia:
    inicializar_episodio()
    estado_actual = estado_inicial
    while episodio_en_curso:
        accion = politica(estado_actual)  # Acción seleccionada según la política
        recompensa, siguiente_estado = ejecutar_accion(accion)  # Ejecución de la acción
        max_q = max(Q[siguiente_estado, :])  # Seleccionar el máximo valor Q en el siguiente estado
        Q[estado_actual, accion] = Q[estado_actual, accion] + alpha * (recompensa + gamma * max_q - Q[estado_actual, accion])
        estado_actual = siguiente_estado
```

## **TD(n) para n>0**

Los algoritmos TD(n) representan una extensión de los métodos de diferencias temporales (TD), que permiten flexibilizar la cantidad de pasos considerados para la actualización de los valores de estado o acción-valor. Esta serie de métodos se sitúa entre los algoritmos TD(0), que utilizan una única transición para la actualización, y los métodos de Monte Carlo, que esperan hasta el final del episodio para realizar una actualización basada en el retorno total.

En los métodos TD(0), como hemos visto, la actualización de la estimación del valor de un estado se basa en la recompensa inmediata más el valor descontado del siguiente estado:

$$
R_t(s_t) \approx r_{t+1} + \gamma \cdot V_t(s_{t+1})
$$

En contraste, los métodos de Monte Carlo utilizan la suma de todas las recompensas del episodio, descontadas apropiadamente:

$$
R_t(s_t) \approx r_{t+1} + \gamma \cdot r_{t+2} + \gamma^2 \cdot r_{t+3} + \gamma^3 \cdot r_{t+4} + \dots
$$

Los algoritmos TD(n) incorporan los principios de ambos enfoques al considerar las recompensas de varios pasos futuros antes de obtener una estimación del valor del estado. Por ejemplo, TD(1), TD(2) y TD(3) extienden el horizonte temporal de las recompensas a considerar:

- **TD(1):**

$$
R_t(s_t)^{(1)} \approx r_{t+1} + \gamma \cdot V_t(s_{t+1})
$$

- **TD(2):**

$$
R_t(s_t)^{(2)} \approx r_{t+1} + \gamma \cdot r_{t+2} + \gamma^2 \cdot V_t(s_{t+2})
$$

- **TD(3):**

$$
R_t(s_t)^{(3)} \approx r_{t+1} + \gamma \cdot r_{t+2} + \gamma^2 \cdot r_{t+3} + \gamma^3 \cdot V_t(s_{t+3})
$$

Estas ecuaciones muestran cómo, en TD(n), el retorno esperado $R_t(s_t)$ se calcula como una combinación de las recompensas recibidas en los siguientes \( n \) pasos más el valor descontado del estado alcanzado después de esos \( n \) pasos. Esto permite una mejor aproximación del retorno verdadero, al incluir más información sobre el futuro inmediato antes de recurrir a una estimación del valor.

## **Métodos on-policy y off-policy**

La filosofía de TD(0) y *Sarsa* es desarrollar muchos episodios para evaluar una política. Una vez evaluada correctamente se actualiza la política y se vuelve a evaluar. Para generar los nuevos episodios se utiliza la política a evaluar. Esto se conoce como métodos **on-policy**. Por el contrario, se denominan métodos **off-policy** a aquellos en los que la actualización de los valores no se basa en la política a evaluar sino en una búsqueda directa de la política óptima, como, por ejemplo, el método *Q-Learning*.


Los métodos **on-policy** tratan de mejorar la política recogiendo datos a los que llega siguiendo su política actual. Los métodos **off-policy** mejoran la política accediendo a estados que no siempre están elegidos por su política. Expliquemos esto un poco mejor. Hemos visto que los métodos que no usan programación dinámica deben explorar estados para acceder a nueva información. Supongamos que elegimos una política $\pi$ que permita cierto grado de exploración, como, por ejemplo,  $\epsilon$-*greedy*. En un algoritmo, como *Q-Learning* o *Sarsa*, con política $\epsilon$-*greedy* la selección del nuevo estado se lleva a cabo haciendo uso de una probabilidad $\epsilon$ (por ejemplo, $\epsilon = 10\%$). Lo que quiere decir que en el 90% de las ocasiones el nuevo estado $s’$ vendrá dado por la acción $a$ con el máximo valor de $Q$ en el estado $s$ (explotación), y un 10% de las veces la acción $a$ se seleccionará aleatoriamente (exploración). 

Una vez tenemos clara la política que vamos a utilizar, tenemos que ver cómo esa política se optimiza. La política mejora a medida que vamos actualizando la tabla $Q$, es decir, tomamos mejores decisiones cuanta mayor información nos da $Q$. Recordemos cómo actualiza la tabla $Q$ el algoritmo *Sarsa*:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot [r + \gamma \cdot Q(s',a') - Q(s,a)]
$$

Y ahora veamos cómo lo hace el algoritmo *Q-Learning*:

$$
Q(s,a) \leftarrow Q(s,a) + \alpha \cdot [r + \gamma \cdot max_{a'}(Q(s',a')) - Q(s,a)]
$$

La diferencia está en que el algoritmo *Q-Learning* ha actualizado $Q$ haciendo uso de la acción $a’$ sobre $s’$ que mayor valor le ofrece. Por tanto, la acción $a’$ no ha venido seleccionada por la política $\pi$, (*off-policy*). Sin embargo, en el algoritmo *Sarsa* la actualización se lleva a cabo por $Q(s’,a’)$ en donde $a’$ sí ha venido dada por la política $\pi$, (*on-policy*).

